# Model Building

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from typing import List

Set a predetermined seed so all our results can be replicated

In [4]:
RANDOM_SEED = 1337

# Preprocessing

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
full_data = pd.read_csv('Dataset/clean_data.csv', index_col='Customer_ID')

In [27]:
full_data = full_data[
    full_data.columns[
        ~full_data.columns.str.contains('kid[0-9]')
    ]
]

In [7]:
#Separate target column with rest of the data
churn_col = full_data['churn'].copy()
full_data = full_data.drop('churn',axis=1)

Since we're finished with EDA in the other notebook, we can start splitting the data into training and testing sets. For models that need validation, we will utilize k-fold CV later on. 

In [28]:
#Separate data for training and testing with 80% for training and 20% testing
#Uses our preselected random seed to results are reproducible 
raw_x_train, raw_x_test, y_train, y_test = train_test_split(
    full_data,
    churn_col,
    test_size=0.2,
    random_state=RANDOM_SEED
)

In [9]:
with open('columnDescriptions.json','r') as f:
    col_desc = json.load(f)
    
#Shortened descriptions with elipses for plot titles
#Only retains first 20 characters of description then appends with elipses
short_col_desc = dict(zip(
    col_desc.keys(),
    map(lambda desc: 
        desc if len(desc)<20 else f'{desc[:20]}...', col_desc.values()
    )
))

We will use several different algorithms then compare their performance afterwards to determine which is the best to use. The algorithm we will use are: 
- Logistic Regression
- K Nearest Neighbor Classifier
- Random Forest
- XGBoost
- LightGBM

Since the implementations selected for the above algorithms have differing aptitudes for missing values and normalization, we will need different preprocessing pipelines of the data. For example, XGBoost and LightGBM can handle nan values (XGBoost learns whether to split nan values during training, while LightGBM allocates nan values to reduce loss afterwards) while the sklearn implementations of Logisitic regression, KNN classifier, and random forest cannot. 

In [30]:
from model_utils import PipelineFactory

In [31]:
pf = PipelineFactory(full_data)
pca_pipe = pf.create_pipe(pca=True,impute=True,normalize=True,pca_comps=32)
# impute_normalize_pipe = pf.create_pipe(impute=True,normalize=True)
# impute_pipe = pf.create_pipe(impute=True,normalize=False)
# ohe_pipe = pf.create_pipe(impute=False,normalize=False)

# Model building

In [41]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgbm
from sklearn.metrics import confusion_matrix, roc_auc_score

## Logistic Regression

In [34]:
x_train = pca_pipe.fit_transform(raw_x_train)
x_test = pca_pipe.transform(raw_x_test)

In [39]:
# L2 regularized logistic regression
# 5 default chosen regularization strength
# 5 fold CV (80% training 20% validation)
log_reg = LogisticRegressionCV(
    Cs=10,
    max_iter=1000,
    random_state=RANDOM_SEED,
    class_weight='balanced'
)

In [40]:
log_reg.fit(x_train,y_train)

LogisticRegressionCV(class_weight='balanced', max_iter=1000, random_state=1337)

In [42]:
roc_auc_score(y_test,log_reg.predict(x_test))

0.5726456975323504

## XGBoost

In [46]:
from xgboost import XGBClassifier

In [49]:
xgb = XGBClassifier(
    random_state=RANDOM_SEED,
    verbosity=2
)

In [50]:
xgb.fit(X=x_train,y=y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto',
              random_state=1337, reg_alpha=0, reg_lambda=1, ...)

In [54]:
roc_auc_score(y_test,xgb.predict(x_test))

0.5841501205411465

In [57]:
xgb.feature_importances_

array([0.01057185, 0.01014336, 0.0141069 , 0.01064938, 0.0094505 ,
       0.0147091 , 0.01204344, 0.01099302, 0.01332961, 0.00959758,
       0.01523669, 0.01057027, 0.00836799, 0.01177141, 0.00780587,
       0.00961335, 0.00903843, 0.00803994, 0.00805278, 0.00804185,
       0.00858402, 0.00972116, 0.00970881, 0.01143767, 0.01037577,
       0.00843853, 0.01408996, 0.0084799 , 0.00853688, 0.00870128,
       0.0093213 , 0.00898632, 0.05368361, 0.00631747, 0.01128681,
       0.00756218, 0.00933052, 0.00897264, 0.01151955, 0.00956161,
       0.00903706, 0.00938401, 0.01245765, 0.00716842, 0.01637909,
       0.00894351, 0.00871176, 0.00726674, 0.01347499, 0.00643387,
       0.00742257, 0.00967339, 0.01889899, 0.01135972, 0.0065766 ,
       0.00964975, 0.0087383 , 0.00853286, 0.0092633 , 0.01281328,
       0.        , 0.01321229, 0.00901223, 0.00936   , 0.00941355,
       0.01455994, 0.        , 0.        , 0.00737674, 0.        ,
       0.        , 0.00776004, 0.        , 0.00686181, 0.00879